In [1]:
# Importing necessary packages and libraries
import numpy as np
import pandas as pd
import random
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [2]:
# Checking the datasets
data = np.load('sdss_galaxy_colors.npy')
df = pd.DataFrame(data)
df.head()

,u,g,r,i,z,spec_class,redshift,redshift_err
0,19.84132,19.52656,19.46946,19.17955,19.10763,b'QSO',0.539301,0.000065
1,19.86318,18.66298,17.84272,17.38978,17.14313,b'GALAXY',0.164570,0.000012
2,19.97362,18.31421,17.47922,17.07440,16.76174,b'GALAXY',0.041900,0.000022
3,19.05989,17.49459,16.59285,16.09412,15.70741,b'GALAXY',0.044277,0.000011
4,19.45567,18.33084,17.67185,17.30189,17.13650,b'GALAXY',0.041644,0.000018


In [3]:
# # From the dataset, defining the color differences as our predictors (i.e. features) and
# the redshift as our response variable (i.e. target).

def get_features_targets(data):
    
    features = np.zeros((data.shape[0], 4))
    features[:, 0] = data['u']-data['g']
    features[:, 1] = data['g']-data['r']
    features[:, 2] = data['r']-data['i']
    features[:, 3] = data['i']-data['z']

    targets = data['redshift']  
    return (features, targets)


In [4]:
# Splitting the dataset into one train and one test subset

def split_data(features, targets):
    split = features.shape[0]//2
    train_features = features[:split]
    test_features = features[split:]
    train_targets = targets[:split]
    test_targets = targets[split:]
    return (train_features, test_features, train_targets, test_targets)

In [5]:
# Calculating the median residual error of our model, 
# i.e. the median of the difference between our predicted and actual redshifts.
# We will use this to test the accuracy and effectivity of our model.

def median_diff(pred, test):
    return np.median(abs(pred-test))

# Calculating the MSE, mean squared error for similar reason

def mse(pred, test):
    return np.mean((pred-test)**2)

In [6]:
## Cheking the accuracy of Decision tree model using one test and one train set


# get the train and test subset
features, targets = get_features_targets(data)
train_features, test_features, train_targets, test_targets = split_data(features, targets)
    
# train the model
dtr = DecisionTreeRegressor()
dtr.fit(train_features, train_targets)
    
# get the predicted_redshifts
predictions = dtr.predict(test_features)

# use median_diff function to calculate the accuracy
print("MSE:", mse(predictions,test_targets))
print("Diff:", median_diff(predictions,test_targets))

MSE: 0.12661433051258347
Diff: 0.021766930000000004


In [7]:
# Estimate the test set MSE with 10-fold cross validation
# w/o randomized fold selection.

n = len(data)
total_mse = 0

for i in range(10):
    test_x = []
    test_y = []
    train_x = []
    train_y = []
    
    # selecting test index
    test = range(i*(int(n/10)),(i+1)*(int(n/10)))
    
    test_x = features[test]
    test_y = targets[test]
    
    for j in range(n):
        if j not in test:
            train_x.append(features[j])
            train_y.append(targets[j])
    
    # train the model
    dtr = DecisionTreeRegressor()
    dtr.fit(train_x, train_y)
    
    # get the predicted_redshifts
    predictions = dtr.predict(test_x)

    # use median_diff function to calculate the accuracy
    MSE = mse(predictions,test_y)
    diff = median_diff(predictions,test_y)
    print("\nModel ",i,":")
    print("MSE:", MSE)
    print("Diff:", diff,'\n')
    
    total_mse = total_mse + MSE
        
print("Cross Validation Estimation of test MSE: ", total_mse/10)


Model  0 :
MSE: 0.15500392779409397
Diff: 0.021203407499999993 


Model  1 :
MSE: 0.17412088605293874
Diff: 0.021462285 


Model  2 :
MSE: 0.12470224836739016
Diff: 0.020519937499999995 


Model  3 :
MSE: 0.12389625357112868
Diff: 0.021592550000000005 


Model  4 :
MSE: 0.11241919495624211
Diff: 0.021427449999999987 


Model  5 :
MSE: 0.16127191918509148
Diff: 0.021692047000000006 


Model  6 :
MSE: 0.10604041804726616
Diff: 0.021910144999999995 


Model  7 :
MSE: 0.13607636600240766
Diff: 0.022226309999999996 


Model  8 :
MSE: 0.10055644020448594
Diff: 0.021989234999999975 


Model  9 :
MSE: 0.11924065327511953
Diff: 0.02206619 

Cross Validation Estimation of test MSE:  0.13133283074561644


In [8]:
# Estimate the test set MSE with 10-fold cross validation
# w/ randomized fold selection.

n = len(data)
total_mse = 0
index = (range(n))

for i in range(10):
    test_x = []
    test_y = []
    train_x = []
    train_y = []

    # selecting test index randomly
    test = random.sample(index,int(n/10))
    # exclude the selected indices from the overall range
    index = list(set(index) - set(test))
    
    test_x = features[test]
    test_y = targets[test]
    
    for j in range(n):
        if j not in test:
            train_x.append(features[j])
            train_y.append(targets[j])
    
    # train the model
    dtr = DecisionTreeRegressor()
    dtr.fit(train_x, train_y)
    
    # get the predicted_redshifts
    predictions = dtr.predict(test_x)

    # use median_diff function to calculate the accuracy
    MSE = mse(predictions,test_y)
    diff = median_diff(predictions,test_y)
    print("\nModel ",i,":")
    print("MSE:", MSE)
    print("Diff:", diff,'\n')
    
    total_mse = total_mse + MSE
        
print("Cross Validation Estimation of test MSE: ", total_mse/10)


Model  0 :
MSE: 0.12111446115999401
Diff: 0.021637335 


Model  1 :
MSE: 0.1281679003893144
Diff: 0.02177569 


Model  2 :
MSE: 0.1448799178910574
Diff: 0.021283425000000016 


Model  3 :
MSE: 0.16217997818905677
Diff: 0.021653572500000003 


Model  4 :
MSE: 0.12174345016584079
Diff: 0.022152635000000004 


Model  5 :
MSE: 0.13930311108155832
Diff: 0.021719299499999997 


Model  6 :
MSE: 0.12344570850346456
Diff: 0.02066087 


Model  7 :
MSE: 0.13297403980018271
Diff: 0.021640155 


Model  8 :
MSE: 0.14423876675371422
Diff: 0.02138696 


Model  9 :
MSE: 0.1156439692928117
Diff: 0.02198377 

Cross Validation Estimation of test MSE:  0.13336913032269948


In [ ]:
# Estimate the test set MSE with k-fold cross validation
# w/ randomized fold selection.

def cross_validation(k):
    if k==0 or k==1:
        return 0
    n = len(data)
    total_mse = 0
    index = (range(n))

    for i in range(k):
        test_x = []
        test_y = []
        train_x = []
        train_y = []

        # selecting test index randomly
        test = random.sample(index,int(n/k))
        # exclude the selected indices from the overall range
        index = list(set(index) - set(test))

        test_x = features[test]
        test_y = targets[test]

        for j in range(n):
            if j not in test:
                train_x.append(features[j])
                train_y.append(targets[j])

        # train the model
        dtr = DecisionTreeRegressor()
        dtr.fit(train_x, train_y)

        # get the predicted_redshifts
        predictions = dtr.predict(test_x)

        # use median_diff function to calculate the accuracy
        MSE = mse(predictions,test_y)
        diff = median_diff(predictions,test_y)
        #print("\nModel ",i,":")
        #print("MSE:", MSE)
        #print("Diff:", diff,'\n')

        total_mse = total_mse + MSE

    print(k,"fold Cross Validation Estimation of test MSE: ", total_mse/k)
    return total_mse/k

In [ ]:
for i in range(10):
    cross_validation(i)

2 fold Cross Validation Estimation of test MSE:  0.12835225766078046
3 fold Cross Validation Estimation of test MSE:  0.1306390393436875
4 fold Cross Validation Estimation of test MSE:  0.12891735076333136
5 fold Cross Validation Estimation of test MSE:  0.1317672009319444
6 fold Cross Validation Estimation of test MSE:  0.13109191597547207
7 fold Cross Validation Estimation of test MSE:  0.1344943038652723
8 fold Cross Validation Estimation of test MSE:  0.1282940100136795
9 fold Cross Validation Estimation of test MSE:  0.13256426445910433


In [ ]:
## Use random forest to create and evaluate new model
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Estimate the test set MSE with k-fold Random Forest cross validation
# w/ randomized fold selection.

def random_forest_cross_validation(k):
    if k==0 or k==1:
        return 0
    n = len(data)
    total_mse = 0
    index = (range(n))

    for i in range(k):
        test_x = []
        test_y = []
        train_x = []
        train_y = []

        # selecting test index randomly
        test = random.sample(index,int(n/k))
        # exclude the selected indices from the overall range
        index = list(set(index) - set(test))

        test_x = features[test]
        test_y = targets[test]

        for j in range(n):
            if j not in test:
                train_x.append(features[j])
                train_y.append(targets[j])

        # train the model
        rfr = RandomForestRegressor(n_estimators = 100)
        rfr.fit(train_x, train_y)

        # get the predicted_redshifts
        predictions = rfr.predict(test_x)

        # use median_diff function to calculate the accuracy
        MSE = mse(predictions,test_y)
        diff = median_diff(predictions,test_y)
        #print("\nModel ",i,":")
        #print("MSE:", MSE)
        #print("Diff:", diff,'\n')

        total_mse = total_mse + MSE

    print(k,"fold Cross Validation Estimation of test MSE: ", total_mse/k)
    return total_mse/k

In [ ]:
random_forest_cross_validation(2)
random_forest_cross_validation(10)

2 fold Cross Validation Estimation of test MSE:  0.07233655955661375
10 fold Cross Validation Estimation of test MSE:  0.07042624555155133


0.07042624555155133

In [ ]:
# Use Random Forest on whole dataset and calculate the R^2 value 
# considering the out of bag (OOB) samples
rfr = RandomForestRegressor(n_estimators = 100, oob_score = True)
rfr.fit(features, targets)
rfr.score(features, targets)